<a href="https://colab.research.google.com/github/nikolay83/MVP_218630/blob/master/TitanicE2E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mp

df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Clean continuous

In [ ]:
#df.isnull().sum()
df['Age'].fillna(df['Age'].mean(), inplace=True)
df.head(10)



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,29.699118,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.000000,1,0,237736,30.0708,NaN,C


In [ ]:
df['FamilyCount'] = df['SibSp'] + df['Parch']
df.drop(['PassengerId', 'SibSp', 'Parch'], axis=1, inplace=True)
df.head(10)
df.to_csv('titanic_clean.csv', index=False)

# Cabin indicator

In [ ]:
df['Cabin_ind'] = np.where(df['Cabin'].isnull, 0, 1)

# Sex to numeric

In [ ]:
gender_num = {'male':0, 'female':1}
df['Sex'] = df['Sex'].map(gender_num)

# Clean non-impactful features

In [ ]:
df.drop(['Cabin', 'Embarked', 'Name', 'Ticket'], axis =1, inplace=True)
df.head()

,Survived,Pclass,Sex,Age,Fare,FamilyCount,Cabin_ind
0,0,3,0,22.0,7.2500,1,0
1,1,1,1,38.0,71.2833,1,0
2,1,3,1,26.0,7.9250,0,0
3,1,1,1,35.0,53.1000,1,0
4,0,3,0,35.0,8.0500,0,0


# Now clean

In [14]:
df=pd.read_csv('titanic_cleaned.csv')

#Split data

In [15]:
from sklearn.model_selection import train_test_split

features = df.drop('Survived', axis=1)
labels = df['Survived']

x_train, x_test, y_train, y_test = train_test_split (features, labels, test_size=0.4, random_state=42)
x_test, x_val, y_test, y_val = train_test_split (x_test, y_test, test_size=0.5, random_state=42)



In [16]:
# and check if the distribution is correct
for dataset in [y_train, y_val, y_test]:
  print(round(len(dataset)/len(labels), 2))

0.6
0.2
0.2


# Fit the model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

tr_features = x_train
tr_labels = y_train

rf = RandomForestClassifier()
scores = cross_val_score(rf, x_train, y_train.values.ravel(), cv=5)
scores


array([0.82242991, 0.8317757 , 0.78504673, 0.81308411, 0.81132075])

# Cross-validation with the grid

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
  'n_estimators': [5, 20, 50, 100],
  'max_depth': [2, 4, 5, 6, 8, 20, None]
}

cv = GridSearchCV(rf, params, cv=5)
cv.fit(x_train, y_train.values.ravel())

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [2, 4, 5, 6, 8, 20, None],
                         'n_estimators': [5, 20, 50, 100]})

In [ ]:
print (cv.best_params_ ,cv.best_score_)

{'max_depth': 8, 'n_estimators': 20} 0.8389878328337153


#Results on validation dataset

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

#fit to the 3 best hyperparameters set
rf1 = RandomForestClassifier(n_estimators=8, max_depth=20)
rf1.fit(x_train, y_train)

rf2 = RandomForestClassifier(n_estimators=5, max_depth=10)
rf2.fit(x_train, y_train)

rf3 = RandomForestClassifier(n_estimators=100, max_depth=10)
rf3.fit(x_train, y_train)

for mdl in [rf1, rf2, rf3]:
  y_pred= mdl.predict(x_val)
  accuracy = round(accuracy_score(y_val, y_pred),3)
  precision = round(precision_score(y_val, y_pred),3)
  recall = round(recall_score(y_val, y_pred),3)
  print('Depth: {} / Estimators: {} => Acc: {} / P: {} / R: {}'.format(
      mdl.max_depth, mdl.n_estimators, accuracy, precision, recall))


Depth: 20 / Estimators: 8 => Acc: 0.81 / P: 0.809 / R: 0.724
Depth: 10 / Estimators: 5 => Acc: 0.804 / P: 0.797 / R: 0.724
Depth: 10 / Estimators: 100 => Acc: 0.832 / P: 0.871 / R: 0.711


#Results on test dataset

In [ ]:
mdl=rf2
y_pred= mdl.predict(x_test)
accuracy = round(accuracy_score(y_test, y_pred),3)
precision = round(precision_score(y_test, y_pred),3)
recall = round(recall_score(y_test, y_pred),3)
print('Depth: {} / Estimators: {} => Acc: {} / P: {} / R: {}'.format(
      mdl.max_depth, mdl.n_estimators, accuracy, precision, recall))

Depth: 10 / Estimators: 5 => Acc: 0.792 / P: 0.769 / R: 0.615


In [ ]:
# prediction output
f=mdl.predict_proba(x_test)
print(f)

# Algo: SVM

In [18]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
svc = SVC()
params = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10]
}

cv = GridSearchCV(svc, params, cv=5)
cv.fit(x_train, y_train)
print (cv.best_params_ ,cv.best_score_)

{'C': 0.1, 'kernel': 'linear'} 0.7959619114794568


In [26]:
#pickle the model
import pickle
with open("SVM_model.pkl", "wb") as f:
    pickle.dump(cv.best_estimator_, f)